---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

def blight_model():
    
    # Your code here
    df_train = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    
    data = df_train[['ticket_id', 'city', 'zip_code', 'violation_code', 'violation_description', 'disposition', 
                     'fine_amount', 'admin_fee', 'state_fee', 'grafitti_status', 'clean_up_cost', 'judgment_amount', 
                     'discount_amount', 'compliance_detail' ]]

    data.set_index('ticket_id', inplace=True)
    data = data[data['compliance_detail'].str.contains('not responsible')==False]
    
    #We are using Complicance Detail as the target: Make this data binary
    data.loc[data['compliance_detail'] != 'compliant by on-time payment', 'compliance_detail'] = 0
    data.loc[data['compliance_detail'] == 'compliant by on-time payment', 'compliance_detail'] = 1

    #Drop Nan rows
    data = data[[      
                    'fine_amount','admin_fee',
                    'state_fee', 'clean_up_cost', 'judgment_amount', 'discount_amount', 'compliance_detail'
                    ]].dropna()
    #Select possible features (I did not use latitude and longitude at all)
    X = data[[      
                    'fine_amount','admin_fee',
                    'state_fee', 'clean_up_cost', 'judgment_amount', 'discount_amount', 
                    ]]
    #Select label (target)
    y = data['compliance_detail']
    
    #Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    y_train = y_train.astype('int')
    
    #Choose Logistic Regression model to start
    LR = LogisticRegression().fit(X_train, y_train)
    grid_values = {'penalty': ['l1', 'l2'], 'C':[0.001, 0.01, 0.1, 1, 10, 100]}

    ##Optimize LR over grid parameters: AUC
    #grid_lr_auc = GridSearchCV(LR, param_grid = grid_values, cv=5, scoring = 'roc_auc')    
    #print('Grid best parameter (max. AUC): ', grid_lr_auc.best_params_)
    #print('Grid best score (AUC): ', grid_lr_auc.best_score_)
    #print('Grid mean score (AUC): ', grid_lr_auc.cv_results_['mean_test_score'].reshape(5,2)) #Choose best among these
    #grid_lr_auc.fit(X_train, y_train) #automatically chooses best model
    #best_model = grid_lr_auc.best_estimator_ #but we can check this as well ourselves
    #y_test = y_test.astype(int)
    #y_pred_best_model = best_model.predict(X_test)
    #auc_best_model = roc_auc_score(y_test, y_pred_best_model)
        
    LR_best = LogisticRegression(penalty='l1', C=0.001).fit(X_train, y_train) #Chosen based on Grid Search best model
    #y_pred = LR_best.predict(X_test)
    #auc = roc_auc_score(y_test, y_pred)
    
    #Try a svm Classifier
    #clf = svm.SVC(kernel='linear') # Linear Kernel
    #clf.fit(X_train, y_train)
    #y_pred = clf.predict(X_test)
    #auc = roc_auc_score(y_test, y_pred)
    
    #Try a KNN Classifier
    #from sklearn.neighbors import KNeighborsClassifier
    #knn = KNeighborsClassifier(n_neighbors=3)
    ## Train the model using the training sets
    #knn.fit(X_train, y_train)
    #y_pred = knn.predict(X_test)
    #auc = roc_auc_score(y_test, y_pred)

    df_test = pd.read_csv('test.csv')
    df_test.set_index('ticket_id', inplace=True)
    X1 = df_test[[  
                    'fine_amount','admin_fee', 
                    'state_fee','clean_up_cost','judgment_amount', 'discount_amount'
                    ]]
    
    #y_score_lr = grid_lr_auc.predict_proba(X1)
    y_score_lr = LR_best.predict_proba(X1)
    #y_score_lr = knn.predict_proba(X1)
    
    return pd.Series(data=y_score_lr[:,1], index=df_test.index)
    
blight_model()

ticket_id
284932    4.136796e-03
285362    6.932143e-09
285361    2.144129e-02
285338    4.136796e-03
285346    2.144129e-02
285345    4.136796e-03
285347    4.791164e-02
285342    1.454874e-01
285530    6.932143e-09
284989    2.830436e-05
285344    4.791164e-02
285343    6.932143e-09
285340    6.932143e-09
285341    4.791164e-02
285349    2.144129e-02
285348    4.136796e-03
284991    2.830436e-05
285532    2.830436e-05
285406    2.830436e-05
285001    2.830436e-05
285006    6.932143e-09
285405    6.932143e-09
285337    2.830436e-05
285496    4.791164e-02
285497    4.136796e-03
285378    6.932143e-09
285589    2.830436e-05
285585    4.136796e-03
285501    2.144129e-02
285581    6.932143e-09
              ...     
376367    4.429619e-07
376366    1.805426e-03
376362    1.805426e-03
376363    4.136796e-03
376365    4.429619e-07
376364    1.805426e-03
376228    1.805426e-03
376265    1.805426e-03
376286    1.579456e-01
376320    1.805426e-03
376314    1.805426e-03
376327    1.579456e-01
3